### Mid-term for HDS5210

Your supervisor is concerned about 4-year survival risks for COPD. She has asked for you to do some analysis using a new metric, BODE. BODE is an improvement on a previous metric and promises to provide insight on survival risks.

BODE is defined here. https://www.mdcalc.com/calc/3916/bode-index-copd-survival#evidence

Your assignment is to create a BODE calculation, use it to calculate BODE scores and BODE survival rates for a group of patients. Then we want to evaluate the average BODE scores and BODE survival rates for each area hospital.

Your patient input file will have the following columns:
NAME,SSN,LANGUAGE,JOB,HEIGHT_M,WEIGHT_KG,fev_pct,dyspnea_description,distance_in_meters,hospital

BODE calculations require a BMI value, so you will have to create a function for it.

Your output should be in the form of two CSV files, patient_output.csv and hospital_output.csv.

Patient_output will have the following columns:
NAME,BODE_SCORE,BODE_RISK,HOSPITAL

Hospital output will have the following columns:
HOSPITAL_NAME, COPD_COUNT, PCT_OF_COPD_CASES_OVER_BEDS, AVG_SCORE, AVG_RISK

Each function you create should have documentation and a suitable number of test cases. If the input data could be wrong, make sure to raise a Value Error.

For this assignment, use the doctest, json, and csv libraries. Pandas is not allowed for this assignment.

In [76]:
import doctest
import json
import csv

### Step 1: Calculate BMI

In [77]:
def calculate_bmi(weight_kg, height_m):
    """
    Calculate the BMI from weight and height.

    >>> calculate_bmi(70, 1.75)
    22.86

    >>> calculate_bmi(60, 1.65)
    22.04

    weight_kg: Weight in kilograms
    height_m: Height in meters
    return: BMI value rounded to two decimal places
    """
    if weight_kg <= 0 or height_m <= 0:
        raise ValueError("Weight and height always must be positive numbers.")

    bmi = weight_kg / (height_m ** 2)
    return round(bmi, 2)

In [78]:
if __name__ == "__main__":
    doctest.testmod()

### Step 2: Calculate BODE Score

In [79]:
def normalize_dyspnea_description(description):
    """
    These are used because the dataset contains these values in the dyspnea_description

    Normalize variations of dyspnea descriptions to fit known categories.

    >>> normalize_dyspnea_description("STOPS AFTER A FEW MINUTES")
    'Severe breathlessness'

    >>> normalize_dyspnea_description("WHEN HURRYING")
    'Moderate breathlessness'
    """
    description = description.upper().strip()
    if "STOPS AFTER A FEW MINUTES" in description:
        return "Severe breathlessness"
    elif "WHEN HURRYING" in description:
        return "Moderate breathlessness"
    elif "UNABLE TO LEAVE HOME" in description:
        return "Severe breathlessness"
    elif "SLOWER THAN PEERS" in description:
        return "Moderate breathlessness"
    elif "WALKING UPHILL" in description:
        return "Moderate breathlessness"
    elif "ONLY STRENUOUS EXERCISE" in description:
        return "Mild breathlessness"
    elif "BREATHLESS WHEN DRESSING" in description:
        return "Severe breathlessness"
    elif "STOPS WHEN WALKING AT PACE" in description:
        return "Severe breathlessness"
    elif "STOPS AFTER 100 YARDS" in description:
        return "Severe breathlessness"
    return description

def calculate_bode_score(bmi, fev_pct, dyspnea_description, distance_in_meters):
    """
    Calculate the BODE score based on BMI, FEV1 percentage, dyspnea description, and distance in meters.

    >>> calculate_bode_score(22, 70, 'ONLY STRENUOUS EXERCISE', 400)
    1
    >>> calculate_bode_score(18, 40, 'STOPS WHEN WALKING AT PACE', 200)
    8
    """
    bode_score = 0

    # Calculate BMI score
    if bmi > 21:
        bode_score += 0
    else:
        bode_score += 1

    # Calculate FEV1 score
    if fev_pct >= 65:
        bode_score += 0
    elif 50 <= fev_pct < 65:
        bode_score += 1
    elif 36 <= fev_pct < 50:
        bode_score += 2
    else:
        bode_score += 3

    # Normalize dyspnea description and map it to a score
    dyspnea_description = normalize_dyspnea_description(dyspnea_description)
    dyspnea_mapping = {
        "No breathlessness": 0,
        "Mild breathlessness": 1,
        "Moderate breathlessness": 2,
        "Severe breathlessness": 3,
    }

    dyspnea_score = dyspnea_mapping.get(dyspnea_description, None)
    if dyspnea_score is None:
        print(f"Invalid dyspnea description: {dyspnea_description}")
        raise ValueError("Invalid dyspnea description.")

    bode_score += dyspnea_score

    # Calculate distance walked score
    if distance_in_meters > 350:
        bode_score += 0
    elif 250 <= distance_in_meters <= 350:
        bode_score += 1
    elif 150 <= distance_in_meters < 250:
        bode_score += 2
    else:
        bode_score += 3

    return bode_score

In [80]:
if __name__ == "__main__":
    doctest.testmod()

### Step 3: Calculate BODE Risk

In [81]:
def calculate_bode_risk(bode_score):
    """
    Calculate the BODE risk based on the BODE score.

    bode_score: BODE score
    return: BODE risk category
    >>> calculate_bode_risk(2)
    80
    >>> calculate_bode_risk(4)
    67
    """
    if bode_score <= 2:
        return 80
    elif 3 <= bode_score <=4:
        return 67
    elif 5<= bode_score <=6:
        return 57
    else:
      return 18

In [82]:
if __name__ == "__main__":
    doctest.testmod()

### Step 4: Load Hospital Data

In [83]:
def load_hospital_data(json_file):
    """
    Load hospital data from a JSON file.
    """
    with open(json_file, 'r') as file:
        return json.load(file)

In [84]:
dyspnea_descriptions = set()

with open('patient.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dyspnea_descriptions.add(row['dyspnea_description'])

# Print the different types of dyspnea descriptions
print("Different types of dyspnea descriptions in the dataset:")
for description in dyspnea_descriptions:
    print(description)

Different types of dyspnea descriptions in the dataset:
WALKING UPHILL
BREATHLESS WHEN DRESSING
WHEN HURRYING
ONLY STRENUOUS EXERCISE
STOPS AFTER 100 YARDS
SLOWER THAN PEERS
STOPS AFTER A FEW MINUTES
UNABLE TO LEAVE HOME
STOPS WHEN WALKING AT PACE


### Step 5: Main business logic

Call BODE Score, BODE Risk functions for each patient.

For each hospital, calculate Avg BODE score and Avg BODE risk and count the number of cases for each hospital.

In [85]:
patient_csv = "patient.csv"
hospital_json = "hospitals.json"

patient_output_file = "patient_output.csv"
hospital_output_file = "hospital_output.csv"

# Load hospital data
hospital_data = load_hospital_data(hospital_json)

# Initialize the hospital metrics dictionary using the hospital names from the JSON data
hospital_metrics = {}

for entry in hospital_data:
    # Iterate over the hospitals list within the entry
    for hospital in entry['hospitals']:
        hospital_metrics[hospital['name']] = {
            'total_bode_score': 0,
            'total_risk': 0,
            'copd_count': 0,
            'beds': hospital['beds']
        }

patient_results = []

# Read patient data from the CSV file
with open(patient_csv, 'r') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        name = row['NAME']
        ssn = row['SSN']
        language = row['LANGUAGE']
        job = row['JOB']
        height_m = float(row['HEIGHT_M'])
        weight_kg = float(row['WEIGHT_KG'])
        fev_pct = float(row['fev_pct'])
        dyspnea_description = row['dyspnea_description']
        distance_in_meters = float(row['distance_in_meters'])
        hospital_name = row['hospital']

        # Calculate BMI, BODE score, and BODE risk
        bmi = calculate_bmi(weight_kg, height_m)
        bode_score = calculate_bode_score(bmi, fev_pct, dyspnea_description, distance_in_meters)
        bode_risk = calculate_bode_risk(bode_score)

        # Add patient results
        patient_results.append([name, bode_score, bode_risk, hospital_name])

        # Update hospital metrics
        if hospital_name in hospital_metrics:
            hospital_metrics[hospital_name]['total_bode_score'] += bode_score
            hospital_metrics[hospital_name]['total_risk'] += bode_risk
            hospital_metrics[hospital_name]['copd_count'] += 1

hospital_output_list = []

# Calculate hospital metrics
for hospital_name, metrics in hospital_metrics.items():
    copd_count = metrics['copd_count']
    if copd_count > 0:
        avg_bode_score = metrics['total_bode_score'] / copd_count
        avg_bode_risk = metrics['total_risk'] / copd_count
    else:
        avg_bode_score = 0
        avg_bode_risk = 0
    pct_of_copd_cases = (copd_count / metrics['beds']) * 100 if metrics['beds'] > 0 else 0
    hospital_output_list.append([hospital_name, copd_count, pct_of_copd_cases, avg_bode_score, avg_bode_risk])

# Write Patient_output.csv
with open(patient_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["NAME", "BODE_SCORE", "BODE_RISK", "HOSPITAL"])
    writer.writerows(patient_results)

# Write Hospital_output.csv
with open(hospital_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["HOSPITAL_NAME", "COPD_COUNT", "PCT_OF_COPD_CASES_OVER_BEDS", "AVG_SCORE", "AVG_RISK"])
    writer.writerows(hospital_output_list)

In [86]:
if __name__ == "__main__":
    doctest.testmod()

In [87]:
import csv
with open('hospital_output.csv') as f:
    reader = csv.reader(f)

    # Iterate through the rows
    for row in reader:
        print(row)


['HOSPITAL_NAME', 'COPD_COUNT', 'PCT_OF_COPD_CASES_OVER_BEDS', 'AVG_SCORE', 'AVG_RISK']
['BJC', '184', '9.2', '5.369565217391305', '51.77717391304348']
['BJC WEST COUNTY', '171', '17.1', '5.286549707602339', '53.380116959064324']
['MISSOURI BAPTIST', '161', '20.125', '5.217391304347826', '52.037267080745345']
['SAINT LOUIS UNIVERSITY', '164', '16.400000000000002', '5.530487804878049', '50.03658536585366']
["ST.MARY'S", '156', '31.2', '5.352564102564102', '53.44871794871795']
["ST.LUKE'S", '164', '20.5', '5.378048780487805', '50.75']


In [88]:
import csv
with open('patient_output.csv') as f:
    reader = csv.reader(f)

    # Iterate through the rows
    for row in reader:
        print(row)

['NAME', 'BODE_SCORE', 'BODE_RISK', 'HOSPITAL']
['Vanessa Roberts', '4', '67', "ST.LUKE'S"]
['Christopher Fox', '5', '57', 'SAINT LOUIS UNIVERSITY']
['Benjamin Johnston', '4', '67', 'BJC']
['Christopher Hernandez', '4', '67', 'MISSOURI BAPTIST']
['Valerie Burch', '2', '80', 'BJC WEST COUNTY']
['Heather Hart', '4', '67', 'SAINT LOUIS UNIVERSITY']
['Ronald Cobb', '6', '57', "ST.MARY'S"]
['Austin French', '8', '18', 'SAINT LOUIS UNIVERSITY']
['Mary Leonard', '8', '18', 'BJC']
['Mrs. Nicole Smith', '6', '57', "ST.MARY'S"]
['Ashley Warren', '8', '18', 'BJC']
['Jeffrey Jacobson', '6', '57', 'BJC WEST COUNTY']
['Angela Bauer', '6', '57', 'BJC WEST COUNTY']
['Jerry Rogers', '5', '57', 'BJC']
['Lisa Beck', '4', '67', 'BJC']
['Bryan Pena', '7', '18', 'SAINT LOUIS UNIVERSITY']
['Jessica Henderson', '4', '67', 'SAINT LOUIS UNIVERSITY']
['Daniel Mitchell', '5', '57', 'MISSOURI BAPTIST']
['Melanie Graham', '4', '67', 'BJC']
['Deborah Jimenez', '6', '57', 'MISSOURI BAPTIST']
['Kathryn Rasmussen', '3'